In [1]:
import json
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

Read in Data Frame

In [2]:
# df = pd.read_csv('sqr_no_comments.csv')

Create a list of all DBN's

In [3]:
# dbn_list = list(df.dbn)

Get the website

In [4]:
# Create an instance of ChromeOptions
options = webdriver.ChromeOptions()
# Run headless mode so browser does not continually open with every iteration
options.add_argument("headless")
# Instatiate chrome driver and pass in the file path to chromedriver
driver = webdriver.Chrome('/Users/kevinmacmat/Desktop/capstone/chromedriver', options=options)

In [5]:
dbn_list = ['29Q034', '01M034', '03M165', '02M347', '03M149', '11X566', '30Q070']

In [6]:
output_list = []

Get comments and output them to output_list

In [7]:
for dbn in dbn_list:
    # Get website 
    driver.get('https://insideschools.org/school/' + dbn)
    # Switch to iframe containing script tag
    driver.switch_to.frame(1)
    text = driver.page_source
    driver.switch_to.default_content()
    # Parse and process the source with BeautifulSoup module by creating an BS object
    soup = BeautifulSoup(text, 'lxml')
    # Access the soup and find the script element's id
    thread = soup.find("script", {"id": "disqus-threadData"})
    # Turn the bs4 tag into a string, remove the script tag, and access the json
    site_json=json.loads(str(thread)[48:-9])
    # Instantiate comments list
    comments_list = []
    # Navigate and loop json, filtering comments by date, to append comments to comments_list
    for comment in site_json['response']['posts']:
        if '2018' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        elif '2019' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        elif '2020' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        else:
            continue
    # Append list of comments to output_list
    output_list.append(comments_list)

In [8]:
# Testing dataframe
df = pd.DataFrame(dbn_list, columns=['dbn'])

In [9]:
# Add comments column to data frame
df['comments'] = output_list

In [10]:
df

,dbn,comments
0,29Q034,[]
1,01M034,[The DOE has tabled a proposal to combine PS 3...
2,03M165,[I'm a parent of a K student in the G&T class ...
3,02M347,"[Rita Fattorusso, an early childhood teacher a..."
4,03M149,"[is claudia agguire still the principal?, No s..."
5,11X566,[Pelham Gardens Middle School is reported to b...
6,30Q070,[sports & arts in school SASF program it so ba...


Export completed data frame

In [ ]:
# df.to_csv('sqr.csv', index=False)